In [1]:
# Core
import numpy as np
import pandas as pd

# Tensorflow / ML
import tensorflow as tf


# Local imports
from asteroid_data import load_ast_elt
from candidate_element import orbital_element_batch, perturb_elts
from ztf_data import load_ztf_nearest_ast, load_ztf_batch
from ztf_data import make_ztf_batch, calc_hit_freq

In [2]:
# Load ztf nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [3]:
ztf_ast

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
0,b'ZTF18acebhfp',676397301515010013,14490,58430.397303,41.357345,58.879488,0.387942,0.653853,0.649598,18.946699,0.865682,1208789,0.005029,41.396388,58.592038,0.390926,0.655662,0.645976
1,b'ZTF18abodmwk',596403415715010014,5831,58350.403414,30.969721,65.305308,0.358224,0.558644,0.748059,19.010401,0.855504,1227812,0.024428,33.729101,64.536183,0.357570,0.578168,0.733393
2,b'ZTF18abodmwk',626428345715010011,10614,58380.428345,30.969705,65.305294,0.358224,0.558644,0.748059,18.935900,0.855504,1169677,0.015510,29.207596,64.817653,0.371401,0.550472,0.747691
3,b'ZTF18abodmwk',630507595715015045,11250,58384.507593,30.969940,65.305305,0.358223,0.558645,0.748059,19.260401,0.855504,1251951,0.012386,30.227911,65.945543,0.352183,0.551505,0.756181
4,b'ZTF18abodmwk',618384965715010022,9040,58372.384965,30.969643,65.305179,0.358226,0.558644,0.748058,19.220200,0.855504,1246591,0.025343,34.169666,64.771024,0.352659,0.579473,0.734740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5697957,b'ZTF20aareruw',1151532523515015015,97109,58905.532523,253.007910,55.485537,-0.165587,-0.169403,0.971537,19.192400,0.608023,1102168,0.036944,253.707834,53.408139,-0.167230,-0.205583,0.964246
5697958,b'ZTF20aarerwx',1151533002615015009,97110,58905.533009,232.886408,53.509617,-0.358833,-0.115301,0.926253,19.687099,0.559474,1028157,0.084402,224.967815,54.919912,-0.406617,-0.047125,0.912382
5697959,b'ZTF20aarerww',1151533002115010003,97110,58905.533009,236.167899,54.618457,-0.322375,-0.116973,0.939357,19.957001,0.392662,539940,0.052254,240.693936,56.155104,-0.272611,-0.115218,0.955200
5697960,b'ZTF20aarervr',1151526063515015015,97098,58905.526065,286.235286,33.876902,0.232120,-0.509626,0.828494,19.049299,0.517241,1246304,0.014054,285.998189,34.657915,0.226704,-0.499251,0.836274


In [4]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top batch_size
idx = np.argsort(hit_count)[::-1]

# Extract the asteroid number and hit count for this batch
ast_num_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [5]:
# Set batch size
batch_size = 1000

# Batch of elements
elts = orbital_element_batch(ast_nums=ast_num_best[0:batch_size])

# Add hit_count column
elts['hit_count'] = hit_count_best[0:batch_size]

In [6]:
# Perturb orbital elements
sigma_a = 0.01 
sigma_e = 0.01 
sigma_f_deg = 1.0
simga_Omega_deg = 1.0
sigma_omega_deg = 1.0
mask_pert = None
random_seed = 42

elts_pert = perturb_elts(elts, sigma_a=sigma_a, sigma_e=sigma_e, sigma_f_deg=sigma_f_deg, 
                         sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg
                         mask_pert=mask_pert, random_seed=random_seed)

In [7]:
# Review perturbed elements
elts_pert

,element_id,a,e,inc,Omega,omega,f,epoch,hit_count
0,51921,2.682597,0.220424,0.499554,4.699703,2.450796,-1.145275,58600.0,194
1,59244,2.631087,0.264941,0.465045,5.738297,1.766995,-1.603885,58600.0,190
2,15786,1.895464,0.047684,0.392360,6.134689,0.804823,-1.259900,58600.0,189
3,3904,2.595620,0.097645,0.261542,5.450163,2.202423,-1.362720,58600.0,188
4,142999,2.613817,0.192717,0.514017,0.238022,0.946463,-1.332351,58600.0,186
...,...,...,...,...,...,...,...,...,...
995,5512,2.272393,0.211886,0.107931,1.240874,4.606670,0.193667,58600.0,109
996,8128,3.185770,0.107892,0.071972,5.705825,4.549087,2.801466,58600.0,109
997,71765,1.943498,0.060930,0.374262,0.301789,1.377122,-2.216101,58600.0,109
998,162351,2.520431,0.240968,0.088282,5.303877,1.616497,-1.344043,58600.0,109


In [8]:
# Arguments to make_ztf_batch
thresh_deg = 10.0
near_ast = False

In [9]:
ztf_elt = load_ztf_batch(elts=elts_pert, thresh_deg=thresh_deg, near_ast=near_ast)